In [1]:

# import os
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:

from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import CharacterTextSplitter
import pandas as pd
import numpy as np

In [3]:
with open('./data/2015-Nissan-Quest-owner-manual.md') as f:
    md_content = f.read()

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header 1"),("##", "Header 2"),("###", "Header 3")])
md_header_splits = markdown_splitter.split_text(md_content)
md_docs = [", ".join(split.metadata.values()) + '\n' + split.page_content for split in md_header_splits]

# excludes = set('Header 1','CAUTION')
# md_header_splits = [split for split in md_header_splits if split.metadata['Header 1'] != 'WARNING']

### Data preprocessing/chunking for sales QA retrieval

In [17]:

import pickle
with open("./data/data.pkl", "rb") as f:
    data = pickle.load(f)
contents, crawl_status = data['contents'],data['crawl_status']

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header 1"),("##", "Header 2"),("###", "Header 3")])
scrapped_chunks = []
for content in contents:
    md_header_splits = markdown_splitter.split_text(content)
    splits_with_header = [ ", ".join(split.metadata.values()) + '\n' + split.page_content for split in md_header_splits]
    scrapped_chunks.extend(splits_with_header)

df1 = pd.read_csv('./data/car_1.csv')
df2 = pd.read_csv('./data/car_2.csv')

car_info_chunks = []
df1 = df1.fillna('')
for i,r in df1.iterrows():
    title = ", ".join([col + ': ' + str(r[col]) for col in df1.columns if col != 'description'])
    des = r.description 
    car_info_chunks.append(title + des)

docs = car_info_chunks + scrapped_chunks

In [2]:
from FlagEmbedding import BGEM3FlagModel, FlagReranker

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like BAAI/bge-m3 is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [23]:
%%time
title_des_embeds = model.encode(docs,batch_size = 64,max_length = 8192,)['dense_vecs']
#des_embeds = model.encode(des,batch_size = 64,max_length = 8192,)['dense_vecs']
np.save("tit_des_embeds.npy", title_des_embeds)

pre tokenize: 100%|██████████| 19/19 [00:00<00:00, 79.18it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 19/19 [00:07<00:00,  2.56it/s]

CPU times: user 12.8 s, sys: 1.27 s, total: 14.1 s
Wall time: 10 s


In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

client = QdrantClient(path="./car/")

In [40]:

# NOTE: consider splitting the data into chunks to avoid hitting the server's payload size limit
# or use `upload_collection` or `upload_points` methods which handle this for you
# WARNING: uploading points one-by-one is not recommended due to requests overhead
client.upsert(
    collection_name="sales_qa",
    points=[
        PointStruct(id=i, vector={ "title_description": tit_des_embed.tolist() },
        payload={'title_description':doc},
        )
        for i, (tit_des_embed, doc) in enumerate(zip(title_des_embeds, docs))
    ] 
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [41]:
query = """断网了怎么办"""
query_embed = model.encode([query])['dense_vecs'][0]

In [42]:
pts = client.query_points(collection_name="test", query=query_embed.tolist(),using="title_description").points

In [2]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [ ]:
from lmdeploy import pipeline
llm = pipeline('TheBloke/LLaMA-7b-AWQ')

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [5]:
import pandas as pd
df1 = pd.read_csv('random_dealership_questions.csv')
df2 = pd.read_csv('dealership_questions.csv')
df = pd.concat([df1[~df1.duplicated()], df2])

In [ ]:
"""Example Inputs & Outputs:

Input: "Do you have any financing deals on SUVs?" → Output: "Sales"
Input: "How much for an oil change on a 2023 Camry?" → Output: "Service"
Input: "What time do you open on Sundays?" → Output: "Other""""

In [11]:
instruction = """Classify customer queries related to a car dealership into three categories:

1. Sales – Queries related to purchasing, financing, trade-ins, vehicle availability, pricing, discounts, or promotions.
2. Service – Queries related to vehicle maintenance, repairs, warranty, recalls, vehicle's owner's manual, service appointments, or roadside assistance.
3. Other – Any query that does not fit into Sales or Service and discusses unrelated topics.

Query: "{}"
Classification:
"""

prompts = [instruction.format(r.Question) for i,r in df.iterrows()]

In [12]:
r = llm(prompts)

2025-02-06 15:05:46,815 - lmdeploy - WARNING - async_engine.py:629 - GenerationConfig: GenerationConfig(n=1, max_new_tokens=512, do_sample=False, top_p=1.0, top_k=50, min_p=0.0, temperature=0.8, repetition_penalty=1.0, ignore_eos=False, random_seed=None, stop_words=None, bad_words=None, stop_token_ids=[151645], bad_token_ids=None, min_new_tokens=None, skip_special_tokens=True, spaces_between_special_tokens=True, logprobs=None, response_format=None, logits_processors=None, output_logits=None, output_last_hidden_state=None)
2025-02-06 15:05:46,818 - lmdeploy - WARNING - async_engine.py:630 - Since v0.6.0, lmdeploy add `do_sample` in GenerationConfig. It defaults to False, meaning greedy decoding. Please set `do_sample=True` if sampling  decoding is needed


In [16]:
df['preds'] = [re.text for re in r]

In [18]:
df[:50]

,Question,preds
0,Is there a returns for this model?,Service
1,Can you tell me about the returns?,Other
2,Is there a interest rate for this model?,Classification: Sales
3,What are the details of the returns?,Classification: Other
4,Do you offer any delivery?,Classification: Other
5,What is the insurance of the car?,Service
6,What is the loan of the car?,Classification: Sales
7,How much is the safety?,Classification: Other
8,What is the mileage of the car?,Classification: Other
9,Can you tell me about the financing?,Classification: Sales
